In [ ]:
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from textblob import TextBlob
tokenizerR = RegexpTokenizer(r'\w+')

In [ ]:
#Read the data and prepocess
df = pd.read_csv('abs.csv')
target = df.iloc[:,0:1].values
text = df.iloc[:,1:2].values
textMask = df.iloc[:,2:3].values
y = df.iloc[:,3:4].values

y = np.reshape(y, len(y))



dflex = pd.read_csv('lexicon.csv')
posLex = dflex.iloc[:,1:2].values
negLex = dflex.iloc[:,0:1].values



def prepreocess(data):

    input_data=[]
    vocab   = []
    for i in data:
        for j in i:
            
            j = j.lower()
            j = j.replace("\n", "")
            j = j.replace('n\'t', 'not')
            j = j.replace('\'ve', 'have')
            j = j.replace('\'ll', 'will')
            j = j.replace('\'re', 'are')
            j = j.replace('\'m', 'am')
            j = j.replace('/', ' / ')
            j = j.replace('-', ' ')
            j = j.replace('!', ' ')
            j = j.replace('?', ' ')
            j = j.replace('+', ' ')
            j = j.replace('*', ' ')
            while "  " in j:
                j = j.replace('  ', ' ')
            while ",," in j:
                j = j.replace(',,', ',')
            j = j.strip()
            j = j.strip('.')
            j = j.strip()
            temp1 = tokenizerR.tokenize(j)


            temp2 = [x for x in temp1 if not x.isdigit()]
            #temp3 = [w for w in temp2 if not w in stop_words]
            input_data.append(temp2)
            for k in temp2:
                vocab.append(k)
    return vocab, input_data

vocab, input_text = prepreocess(text)
temp , input_target = prepreocess(textMask)

def prepreocess_lexicon(data):
    lexVocab = []
    for i in data:
        for j in i:
            lexVocab.append(j)

    return lexVocab


pos = prepreocess_lexicon(posLex)
neg = prepreocess_lexicon(negLex)

def lexicon_based(data1):
    for i in data1:
        for j in range(len(i)):
            if i[j] in pos[:]:
                i[j] = 'positive' 
            if i[j] in neg[:]:
                i[j] = 'negative'
    return data1


In [ ]:
#Prerpocess
lexicon_input = lexicon_based(input_text)
lexicon_target = lexicon_based(input_target)

#Label Encoder
from keras.utils.np_utils import to_categorical
categorical_labels = to_categorical(y, num_classes=3)

In [ ]:
#Tokenize and Split datasets
from keras.preprocessing import text, sequence
def make_df(train_path,train_path_mask, max_features, maxlen, as_max_len, label):
    tokenizer1 = text.Tokenizer(num_words=max_features)
    tokenizer1.fit_on_texts(train_path)
    tokenizer1.fit_on_texts(train_path_mask)
    list_tokenized_train = tokenizer1.texts_to_sequences(train_path)
    print(tokenizer1)
    data = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen, padding='post')
    list_tokenized_train1 = tokenizer1.texts_to_sequences(train_path_mask)
    data1 = sequence.pad_sequences(list_tokenized_train1, maxlen=as_max_len, padding='post')
    X_t = data[:3608,:]
    X_te = data[3608:,:]
    ytr = label[0:3608,:]
    yte = label[3608:,:]
    X_t_mask = data1[:3608,:]
    X_te_mask = data1[3608:,:]

    word_index = tokenizer1.word_index
    print(word_index)
    return X_t, X_te, X_t_mask, X_te_mask, ytr, yte, word_index

In [ ]:
#Encode the Glove embedding into Input
import numpy as np
def make_glovevec(glovepath, max_features, embed_size, word_index, veclen=300):
    embeddings_index = {}
    f = open(glovepath)
    for line in f:
        values = line.split()
        word = ' '.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs.reshape(-1)
    f.close()

    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
#Download Glove Embedding
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

In [ ]:

#unzip
!unzip glove.6B.zip

In [ ]:

#Call the required  function
max_features = 3600
maxlen = 85
embed_size =300
as_max_len = 85
    
xtr, xte, xtr_mask, xte_mask, ytr,yte,  word_index = make_df(lexicon_input,lexicon_target, max_features, maxlen,as_max_len, categorical_labels)
embedding_vector = make_glovevec("glove.42B.300d.txt", max_features, embed_size, word_index)



In [ ]:
from keras.layers import Layer, Input, Embedding, Bidirectional, LSTM, Concatenate, Dense
import keras.backend as K
from keras.layers import *
from keras import regularizers
from keras import initializers
from keras.models import Model
from keras.optimizers import Adam



In [ ]:
# Attention Layer
class Attention(Layer):
    def __init__(self,**kwargs):
        self.supports_masking = True
        self.attention_dim = 75
        super(Attention,self).__init__(**kwargs)



    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal", trainable = True)
        self.b=self.add_weight(name="att_bias",shape=(self.attention_dim,),initializer="normal", trainable = True)
        self.u=self.add_weight(name="u_bias",shape=(self.attention_dim,1),initializer="normal", trainable = True)        
        super(Attention, self).build(input_shape)


    def compute_mask(self, inputs, mask=None):
        return mask


    def call(self,x, mask=None):
        et= K.tanh(K.dot(x,self.W)+self.b)
        ait = K.dot(et, self.u)
        ait = K.squeeze(ait, -1)		
        ait = K.exp(ait)

        if mask is not None:
            ait *= K.cast(mask, K.floatx())
            
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)
        return output

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [ ]:
#Main BiRU based Model
word_attention_model = None
def build_model(VOCABULARY_SIZE, MAX_SENTENCE_LENGTH, ASP_MAX_LEN, embedding_weights, n_classes=3, embedding_dim=300):

    l2_reg = regularizers.l2(0.001)
    sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), name="input_1")
    aspect_in = Input(shape=(ASP_MAX_LEN,), name="input_2")
    embedding_trainable = False
    embedded_word_seq = Embedding(VOCABULARY_SIZE,embedding_dim,input_length=MAX_SENTENCE_LENGTH,weights=[embedding_weights],trainable=False,mask_zero=False)(sentence_in)

    word_encoder = Bidirectional(GRU(300,return_sequences=True, dropout=0.5))(embedded_word_seq)
    
    #dense_transform_word = Dense(100, activation='relu', name='dense_transform_word', kernel_regularizer=l2_reg)(word_encoder)

    embedded_aspect = Embedding(VOCABULARY_SIZE,embedding_dim,input_length=ASP_MAX_LEN,weights=[embedding_weights],trainable=False,mask_zero=False)(aspect_in)

    aspect_encoder = Bidirectional(GRU(300, return_sequences=True, dropout=0.5))(embedded_aspect)

    #dropout1 = Dropout(0.5)(aspect_encoder)

    #dense_transform_word1 = Dense(100, activation='relu', name='dense_transform_word1', kernel_regularizer=l2_reg)(aspect_encoder)

    attention_weighted_text1 = Attention(name="sentence_attention1")(aspect_encoder)


    attention_weighted_text = Attention(name="sentence_attention")(word_encoder)
  
    attention_fusion = Concatenate()([attention_weighted_text, attention_weighted_text1])

  

    prediction = Dense(n_classes, activation='softmax')(attention_fusion)

    model = Model([sentence_in, aspect_in], prediction)


    optimizer=Adam(lr=0.001, decay=0.0001)

    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
#Run Experiments
K.clear_session()
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
model = build_model(max_features, as_max_len, as_max_len, embedding_vector, 3, embed_size)

#filepath="weights.best.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

#model.fit([xtr, xtr_mask], ytr, validation_data=([xte,xte_mask], yte), batch_size=64, epochs=100,callbacks=[checkpoint], verbose=1)
#csv_logger = CSVLogger("res15(0.1).csv", append=True)
model.fit([xtr, xtr_mask], ytr, validation_data=([xte,xte_mask], yte), batch_size=64, epochs=50, verbose=1)




In [ ]:
from keras.models import load_model
from keras.utils import CustomObjectScope
model = load_model("weights.best.hdf5",custom_objects={'Attention': Attention})
model.summary()
# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([xte, xte_mask], yte, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
ypred = model.predict([xte, xte_mask])
y_pred_class = ypred.argmax(axis=-1)
ytrue = yte.argmax(axis=-1)
from sklearn.metrics import f1_score

F1 = f1_score(ytrue, y_pred_class, average='macro')
print(F1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytrue, y_pred_class)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt='g'); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
#ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['neg', 'neu', 'pos']); ax.yaxis.set_ticklabels(['neg', 'neu', 'pos']);
plt.savefig('confusion_matrix'+' res16'+'.pdf', bbox_inches = "tight")

In [ ]:
model = load_model("weights.best.hdf5",custom_objects={'Attention': Attention})
model = Model(inputs=model.input,outputs=[model.output, model.get_layer('bidirectional').output])
attn = model.get_layer('sentence_attention').output
encoded_input_text = [xte[0:1:,:],xte_mask[0:1,:]]
outputs_bfr_attn = model.predict(encoded_input_text)
weight = outputs_bfr_attn[1]
weight.shape

(1, 80, 600)

In [ ]:
model = load_model("weights.best.hdf5",custom_objects={'Attention': Attention})
encoded_input_text = [xte[179:180:,:],xte_mask[179:180,:]]
layer_weights = model.layers[6].get_weights()
new_model = Model(inputs=model.input, outputs=model.layers[4].output)
output_before_att = new_model.predict(encoded_input_text) #extract layer output
eij = np.tanh(np.dot(output_before_att, layer_weights[0]) + layer_weights[1]) #Eq.(5) in the paper
eij = np.dot(eij, layer_weights[2]) #Eq.(6)

eij = eij.reshape((eij.shape[0], eij.shape[1])) # reshape the vector

ai = np.exp(eij) #Eq.(6)

weights = ai / np.sum(ai) # Eq.(6)
weights

In [ ]:
from IPython.core.display import display, HTML
from string import Template
import json

def display_attention(sentence,attention_weights,scale=0,offset=18,style=1):

	if( len(sentence)!=len(attention_weights)):
		raise Exception("Number of tokens "+ str(len(sentence)) + " not equal to attention list length " + str(len(attention_weights)))

	for _ in attention_weights:
		if _ >1.0 or _ < 0.0:
			raise Exception("invalid value " + str(_) + "in attention_weights")

	html_template = Template('''
	<script src="https://d3js.org/d3.v4.min.js"></script>
	<style>$css_text</style>
	<script>$js_text</script>
	<div id = 'text' style="margin-left:50px;"></div>
	''')

	css_text = '''
	html, body {
	margin: 0;
	padding: 0;
	}
	.tooltip {
	position: relative;
	display: inline-block;
	border-bottom: 1px dotted black;
	}
	.tooltip:hover .tooltiptext {
	visibility: visible;
	}
	.tooltip .tooltiptext {
	visibility: hidden;
	font-size:15px;
	width: 60px;
	background-color: black;
	color: #fff;
	text-align: center;
	border-radius: 6px;
	padding: 1px 0;
	position: absolute;
	left: 50%;
	margin-left: -60px;
	}
	'''

	js_template = Template('''
	var dataset =[["This",0.4],["is",0.3],["a",0.2],["text",0.4]]
	var seconddataset = ["Hello","World"]
	function float2int (value) {
	return value | 0;
	}
	function tohex(fraction)
	{
	var value = float2int(255 * fraction);
	if(value==0)
	return "00"
	var mapping = ['A','B','C','D','E','F'];
	var hex="";
	while(value!==0)
	{
	var curr= value%16;
	if(curr >9)
	  {
	    hex = mapping[curr-10] + hex;
	  }
	else
	  hex = curr + hex;
	value = float2int(value/16);
	}
	return hex;
	}
	d3.select('#text')
	.selectAll('#text')
	.data($data)
	.enter()
	.append('tspan')
	.style('font-family','verdana')
	$style
	.style('margin','2px')
	.attr("class","tooltip")
	.style('font-size',function(d){return  $offset +$scale*d.weight + "px" ;})
	.attr("onmouseover", "handleMouseOver()")
	.text(function(d){
	return d.token+" " ;
	})
	.append('span')
	.attr('class',"tooltiptext")
	.text(function(d){
	return Math.round(d.weight*10000)/100;
	});
	''')

	style1 =".style('background-color', function(d,i){return '#FF' +tohex(1-d.weight) +tohex(1-d.weight) ;})"
	data = [{'token':token, 'weight' :weight} for token,weight in zip(sentence,attention_weights)]

	js_text = js_template.substitute({'data' : json.dumps(data),'scale': 0 if style==1 else scale, 'offset':offset, 'style': style1 if style==1 else ""})


	return display(HTML(html_template.substitute({'css_text':css_text,'js_text':js_text})))


sentence = ['the',
 'food',
 'is',
 'here',
 'is',
 'positive',
 'though',
 'the',
 'quality',
 'is',
 'negative',
 'during',
 'lunch']
attention_weights = [0.00020332, 0.08182091, 0.00613416, 0.00028234, 0.08213249,
        0.08816207, 0.08652914, 0.00445198, 0.08577133, 0.08296202,
        0.08816148, 0.07696584, 0.08714949]

result = display_attention(sentence, attention_weights)


In [ ]:
import keras
from keras.layers import Dense, Embedding, Bidirectional, GRU, TimeDistributed, Input, Dropout, Lambda
from keras.models import Model
from keras import backend as K
import numpy as np

EMBEDDING_DIM = 300
POSITION_EMBEDDING_DIM = 50
MAX_LEN = 80


def reduce_dimension(x, length, mask):
    res = K.reshape(x, [-1, length])  # (?, 78)
    res = K.softmax(res)
    res = res * K.cast(mask, dtype='float32')  # (?, 78)
    temp = K.sum(res, axis=1, keepdims=True)  # (?, 1)
    temp = K.repeat_elements(temp, rep=length, axis=1)  # (?, 78)
    return res / temp


def reduce_dimension_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3  # only valid for 3D tensors
    return [shape[0], shape[1]]


def attention(x, dim):
    res = K.batch_dot(x[0], x[1], axes=[1, 1])
    return K.reshape(res, [-1, dim])


def attention_output_shape(input_shape):
    shape = list(input_shape[1])
    assert len(shape) == 3
    return [shape[0], shape[2]]


def no_change(input_shape):
    return input_shape


def liter(x, length):
    res = K.repeat(x, length)  # (?, 82, 300)
    return res


def liter_output_shape(input_shape):
    shape = list(input_shape)
    return [shape[0], MAX_LEN, shape[1]]


def build_model(max_len=80, aspect_max_len=80, embedding_matrix=[],
                position_embedding_matrix=[], class_num=3, num_words=3700):

    MAX_LEN = max_len
    # ============================= Input =================================================
    sentence_input = Input(shape=(max_len,), dtype='int32', name='sentence_input')  # (?, 78)
    position_input = Input(shape=(max_len,), dtype='int32', name='position_input')  # (?, 78)
    aspect_input = Input(shape=(aspect_max_len,), dtype='int32', name='aspect_input')  # (?, 78)

    sentence_embedding_layer = Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix],
                                         input_length=max_len, trainable=False, mask_zero=True)
    sentence_embedding = sentence_embedding_layer(sentence_input)  # (?, 78, 300)

    position_embedding = Embedding(max_len * 2, POSITION_EMBEDDING_DIM, weights=[position_embedding_matrix],
                                   input_length=max_len, trainable=True, mask_zero=True)(position_input)  # (?, 78, 50)

    aspect_embedding_layer = Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix],
                                       input_length=aspect_max_len, trainable=False, mask_zero=True)
    aspect_embedding = aspect_embedding_layer(aspect_input)  # (?, 9, 300)
    # ============================= Input =================================================

    # =========================== GRU Layer ===============================================
    input_embedding = keras.layers.concatenate([sentence_embedding, position_embedding])  # (?, 78, 350)
    encode_x = Bidirectional(GRU(300, activation="relu", return_sequences=True, recurrent_dropout=0.5, dropout=0.5))(input_embedding)  # (?, 78, 600)
    # =========================== GRU Layer ===============================================

    # =========================== attention ===============================================

    # --------------------------- source2aspect attention ----------------------------------------
    aspect_attention = TimeDistributed(Dense(1, activation='tanh'))(aspect_embedding)  # (?, 9, 1)
    aspect_attention = Lambda(reduce_dimension,
                              output_shape=reduce_dimension_output_shape,
                              arguments={'length': aspect_max_len},
                              mask=aspect_embedding_layer.get_output_mask_at(0),
                              name='aspect_attention')(aspect_attention)  # (?, 9)
    aspect_embedding = Lambda(attention,
                              output_shape=attention_output_shape,
                              arguments={'dim': 300})([aspect_attention, aspect_embedding])  # (?, 300)
    # --------------------------- aspect attention ----------------------------------------

    aspect_embedding = Lambda(liter,
                              output_shape=liter_output_shape,
                              arguments={'length': max_len})(aspect_embedding)  # (?, 82, 300)
    x = keras.layers.concatenate([aspect_embedding, encode_x])  # (?, 82, 900)
    x = TimeDistributed(Dense(300, activation='tanh'))(x)   # (?, 82, 300)
    x = keras.layers.concatenate([x, encode_x])  # (?, 82, 900)

    # --------------------------- source2token attention ----------------------------------------
    x = TimeDistributed(Dense(1, activation='tanh'))(x)  # (?, 78, 1)
    attention_x = Lambda(reduce_dimension,
                         output_shape=reduce_dimension_output_shape,
                         arguments={'length': max_len},
                         mask=sentence_embedding_layer.get_output_mask_at(0),
                         name='attention_x')(x)  # (?, 78)
    # --------------------------- source2token attention ----------------------------------

    # =========================== attention ===============================================

    x = Lambda(attention, output_shape=attention_output_shape, arguments={'dim': 600})([attention_x, encode_x])  # (?, 600)

    x = Dropout(rate=0.5)(x)
    predictions = Dense(class_num, activation='softmax')(x)  # (?, 3)

    model = Model(inputs=[sentence_input, position_input, aspect_input], outputs=predictions)
    model.compile(loss=['categorical_crossentropy'], optimizer='rmsprop', metrics=['accuracy'])

    print(model.summary())
    return model


In [ ]:
def load_position_matrix():
        position_matrix_file = "position_matrix.txt"
        rf = open(position_matrix_file, 'r')
        position_matrix = []
        while True:
            line = rf.readline()
            if line == "":
                break
            temp = line.strip().split()
            for i in range(len(temp)):
                temp[i] = float(temp[i])
            position_matrix.append(temp)
        rf.close()
        position_matrix = np.array(position_matrix)
        return position_matrix

position_matrix= load_position_matrix()
position_matrix.shape

(164, 50)

In [ ]:

model = build_model(maxlen, as_max_len, embedding_vector, position_matrix,3, embed_size)
model.fit([xtr, xtr_mask], ytr, batch_size=64, epochs=100, verbose=1)